# To do/consider/fix:
- Eventually make your code Object oriented?
- Are Jupyter notebooks analyzed in the same way??
- problem in the 'from keras import X, Y, Z', sometimes multiple imports in one line, can't just take last one
- Remove all comments from the text? Pretty easy - If you find '#' delete text until '/n'
- LDA: Convert packages to topics

### Metrics to measure
- number of classes and functions (and what fraction of their code is in functions/classes?)

In [104]:
import requests
from requests.auth import HTTPBasicAuth
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
import operator
import numpy as np
import re
%matplotlib inline

auth = open("utils/auth.txt").read()
username, pw = auth.split()[0], auth.split()[1]

## Links
https://stackoverflow.com/questions/10625190/most-suitable-python-library-for-github-api-v3 - PyGithub  
https://stackoverflow.com/questions/42801033/how-to-make-an-url-with-get-repos-owner-repo-labels-name - how to make url from Github GET request
https://stackoverflow.com/questions/43259946/github-api-getting-topics-of-a-github-repository - getting topics  
http://docs.python-requests.org/en/master/user/advanced/ - headers  
http://docs.python-requests.org/en/master/user/authentication/ - Requests Authentication   
https://github.com/nelsonic/github-scraper - go to "Interesting Facts"  
https://developer.github.com/v3/#pagination - need to access multiple pages

https://developer.github.com/v3/repos/#get - github API get request  
https://developer.github.com/v3/search/#search-topics - Github search repos by topic   
https://developer.github.com/v3/guides/traversing-with-pagination/ - Github pagination   
https://stackoverflow.com/questions/33878019/how-to-get-data-from-all-pages-in-github-api-with-python - pagination get next page
https://stackoverflow.com/questions/48390046/download-github-repositories-list-matching-a-topic - search github repos containing a specific topic  

http://docs.python-requests.org/en/v0.10.6/api/ - Requests API

In [2]:
class Github_Profile:
    def __init__(self, pckgs):
        self.user = ''
        self.url = ''
        
        # metrics
        self.classes = 0
        self.defs = 0
        self.packages = {}
        for p in pckgs:
            self.packages[p] = 0

In [3]:
# get frequency of desired packages in python code
def get_package_freq(text, GProfile):
    for p in GProfile.packages.keys():
        if text.find(p) != -1:    # is package even in script at all?
    
            # direct import, e.g. 'import numpy (as np)'
            string = 'import %s'%p
            loc = text.find(string)
            if loc != -1:
                # last word in line is call_name
                call_name = text[loc:].partition('\n')[0].split()[-1].split('\n')[0].split('\\')[0]
                GProfile.packages[p] += text.count('%s.'%call_name)

            # indirect import, e.g. 'from numpy.X import vectorize (as vec)'
            string_from = 'from %s'%p
            loc_from = text.find(string_from)
            ext = ['(', '.']  # check both X() and X.y()
            while loc_from != -1: # iterate same package imported mult. times on different lines 
                try:
                    call_names = text[loc_from:].partition('\n')[0].split(' import ')[1]
                    if call_names.count(',') != 0:  # iterate mulitple imports on same line
                        call_names = call_names.replace('(','').replace(')','').replace(" ", "").split(',')
                        for call_name in call_names:
                            if call_name == "": # skip bad entries
                                continue
                            GProfile.packages[p] += sum(text.count('%s%s'%(call_name, x)) for x in ext) 
                    else:
                        call_name = call_names.split()[-1]
                        GProfile.packages[p] += sum(text.count('%s%s'%(call_name, x)) for x in ext)
                except:
                    pass
                loc_from = text.find(string_from, loc_from + 1) # get next instance, e.g. 'from numpy.X import b'

def update_metrics(item, GProfile):
    text = requests.get(item['download_url'],
                        headers={"Accept":"application/vnd.github.mercy-preview+json"},
                        auth=HTTPBasicAuth(username, pw)).text
    
    #metrics
    get_package_freq(text, GProfile)
    #get_def_class_freq(text, GProfile)  #still need to create this
    
                    
# recursive, main functions
def digest_repo(repo_url, GProfile):
    r = requests.get('%s'%repo_url, headers={"Accept":"application/vnd.github.mercy-preview+json"},
                 auth=HTTPBasicAuth(username, pw))
    repoItems = json.loads(r.text or r.content)

    for item in repoItems:
        if (item['type'] == 'file' and (item['name'][-3:] == '.py' or item['name'][-6:] == '.ipynb') 
            and 'ipynb_checkpoints' not in item['download_url']):
            print(item['download_url'])
            update_metrics(item, GProfile)
        elif item['type'] == 'dir':
            digest_repo(item['url'], GProfile)

In [8]:
text = requests.get('https://raw.githubusercontent.com/silburt/DeepMoon/master/model_train.py', headers={"Accept":"application/vnd.github.mercy-preview+json"},
                 auth=HTTPBasicAuth(username, pw)).text

In [118]:
def filter_characters(value):
    allowed_characters = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    return ''.join(c for c in value if c in allowed_characters )

filter_characters(text).split()

['usrbinenv',
 'pythonConvolutional',
 'Neural',
 'Network',
 'Training',
 'FunctionsFunctions',
 'for',
 'building',
 'and',
 'training',
 'a',
 'UNET',
 'Convolutional',
 'Neural',
 'Network',
 'onimages',
 'of',
 'the',
 'Moon',
 'and',
 'binary',
 'ring',
 'targetsfrom',
 'future',
 'import',
 'absoluteimport',
 'division',
 'printfunctionimport',
 'numpy',
 'as',
 'npimport',
 'pandas',
 'as',
 'pdimport',
 'hpyfrom',
 'kerasmodels',
 'import',
 'Modelfrom',
 'keraslayerscore',
 'import',
 'Dropout',
 'Reshapefrom',
 'kerasregularizers',
 'import',
 'lfrom',
 'kerasoptimizers',
 'import',
 'Adamfrom',
 'kerascallbacks',
 'import',
 'EarlyStoppingfrom',
 'keras',
 'import',
 'backend',
 'as',
 'KKsetimagedimorderingtfimport',
 'utilstemplatematchtarget',
 'as',
 'tmtimport',
 'utilsprocessing',
 'as',
 'proc',
 'Check',
 'Keras',
 'version',
 'code',
 'will',
 'switch',
 'API',
 'if',
 'neededfrom',
 'keras',
 'import',
 'version',
 'as',
 'kerasversionk',
 'True',
 'if',
 'kerasve

In [101]:
for sym_s, sym_e in [('#','\n'), ('"""', '"""')]:
    start = text.find(sym_s)
    end = text.find(sym_e, start + 1)
    while start != -1:
        print(text[start: end].split())
        start = text.find(sym_s, end + 1)
        end = text.find(sym_e, start + 1)

['#!/usr/bin/env', 'python']
['#', 'Check', 'Keras', 'version', '-', 'code', 'will', 'switch', 'API', 'if', 'needed.']
['#', 'If', 'Keras', 'is', 'v2.x.x,', 'create', 'Keras', '1-syntax', 'wrappers.']
['########################']
['########################']
['#', 'Random', 'color', 'inversion']
['#', 'for', 'j', 'in', 'np.where(np.random.randint(0,', '2,', 'batch_size)', '==', '1)[0]:']
['#', 'd[j][d[j]', '>', '0.]', '=', '1.', '-', 'd[j][d[j]', '>', '0.]']
['#', 'Horizontal/vertical', 'flips']
['#', 'left/right']
['#', 'up/down']
['#', 'Random', 'up/down', '&', 'left/right', 'pixel', 'shifts,', '90', 'degree', 'rotations']
['#', 'Horizontal', 'shift']
['#', 'Vertical', 'shift']
['#', '90', 'degree', 'rotations']
['########################']
['#', 'Get', 'csvs', 'of', 'human-counted', 'craters']
['#', 'remove', 'small/large/half', 'craters']
['#', 'Exclude', 'csvs', 'with', 'few', 'craters']
['#', 'Calculate', 'custom', 'metrics']
['########################']
['#', 'Final', 'output']


In [95]:
n_comments = 0
loc = text.find('#')
endline = text[loc + 1:].find('\n')
while loc != -1:
    print(text[loc: endline], max(len(text[loc: endline].split()) - 1, 0))
    print('----------------------------------')
    n_comments += max(len(text[loc: endline].split()) - 1, 0)
    loc = text.find('#', loc + 1)
    endline = text.find('\n', loc)

#!/usr/bin/env pytho 1
----------------------------------
# Check Keras version - code will switch API if needed. 10
----------------------------------
# If Keras is v2.x.x, create Keras 1-syntax wrappers. 8
----------------------------------
######################## 0
----------------------------------
####################### 0
----------------------------------
###################### 0
----------------------------------
##################### 0
----------------------------------
#################### 0
----------------------------------
################### 0
----------------------------------
################## 0
----------------------------------
################# 0
----------------------------------
################ 0
----------------------------------
############### 0
----------------------------------
############## 0
----------------------------------
############# 0
----------------------------------
############ 0
----------------------------------
########### 0
---------------

## Main execution function

In [52]:
users = ['silburt']#, 'cczhu']
packages = ['numpy', 'pandas', 'keras', 'sklearn', 'matplotlib']

Statistics = []
for u in users:
    GProfile = Github_Profile(packages)
    GProfile.url = 'https://api.github.com/users/%s/repos'%u
    r = requests.get(GProfile.url, headers={"Accept":"application/vnd.github.mercy-preview+json"},
                     auth=HTTPBasicAuth(username, pw))
    userItems = json.loads(r.text or r.content)

    for item in userItems:
        if item['fork'] == False:  # for now ignore forks
            repo_url = '%s/contents'%item['url']
            digest_repo(repo_url, GProfile)
    print(GProfile.packages)
    Statistics.append(GProfile)

https://raw.githubusercontent.com/silburt/astrostats_workshop/master/Stability_Workbook.ipynb
https://raw.githubusercontent.com/silburt/DeepMoon/master/docs/Using%20Zenodo%20Data.ipynb
https://raw.githubusercontent.com/silburt/DeepMoon/master/get_unique_craters.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/input_data_gen.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/model_train.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/run_get_unique_craters.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/run_input_data_gen.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/run_model_train.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/tests/test_get_unique_craters.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/tests/test_input_data_gen.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/tests/test_model_train.py
https://raw.githubusercontent.com/silburt/DeepMoon/master/tests/test_utils.py
https

https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC_push/Day2/Ari_Titanic.ipynb
https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC_push/Day2/Decision_trees_forests.ipynb
https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC_push/Day2/SVC-basic.ipynb
https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC_push/Day2/TitanicPandas.ipynb
https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC_push/Day2/Titanic_RF_Intermediate.ipynb
https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC_push/Day2/Titanic_RF_Simple.ipynb
https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC_push/Day2/Titanic_RF_advanced.ipynb
https://raw.githubusercontent.com/silburt/MachineLearningScratchRepo/master/MachineLearningUTSC

https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/Forces.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/FourierSpectrum.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/Horizons.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/Megno.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/OrbitPlot.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/OrbitalElements.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/ParticleIDsAndRemoval.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/PoincareMap.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/SaturnsRings.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython_examples/Testparticles.ipynb
https://raw.githubusercontent.com/silburt/rebound2/master/ipython

## ------------------------ Scratch ------------------------

In [97]:
users = ['silburt']#, 'cczhu']

for u in users:
    U = User()
    U.url = 'https://api.github.com/users/%s/repos'%u
    r = requests.get(U.url, headers={"Accept":"application/vnd.github.mercy-preview+json"},
                     auth=HTTPBasicAuth(username, pw))
    userItems = json.loads(r.text or r.content)
    
    metrics = initialize_metrics()
    for item in userItems:
        repo_url = '%s/contents'%item['url']
        print(repo_url)
        metrics = digest_repo(repo_url, metrics)
    print(metrics)

https://api.github.com/repos/silburt/astrostats_workshop/contents
https://api.github.com/repos/silburt/cv_resume/contents
https://api.github.com/repos/silburt/DeepMoon/contents
{'numpy': 16, 'keras': 1, 'pandas': 0} get_unique_craters.py
{'numpy': 51, 'keras': 1, 'pandas': 5} input_data_gen.py
{'numpy': 93, 'keras': 56, 'pandas': 8} model_train.py
{'numpy': 94, 'keras': 56, 'pandas': 8} run_get_unique_craters.py
{'numpy': 94, 'keras': 56, 'pandas': 8} run_input_data_gen.py
{'numpy': 94, 'keras': 56, 'pandas': 8} run_model_train.py
{'numpy': 104, 'keras': 56, 'pandas': 9} test_get_unique_craters.py
{'numpy': 164, 'keras': 56, 'pandas': 15} test_input_data_gen.py
{'numpy': 166, 'keras': 58, 'pandas': 15} test_model_train.py
{'numpy': 188, 'keras': 58, 'pandas': 15} test_utils.py
{'numpy': 188, 'keras': 58, 'pandas': 15} __init__.py
{'numpy': 190, 'keras': 58, 'pandas': 15} processing.py
{'numpy': 209, 'keras': 58, 'pandas': 15} template_match_target.py
{'numpy': 210, 'keras': 58, 'pandas

{'numpy': 923, 'keras': 2537, 'pandas': 27} data_utils.py
{'numpy': 927, 'keras': 2537, 'pandas': 27} generic_utils.py
{'numpy': 931, 'keras': 2537, 'pandas': 27} io_utils.py
{'numpy': 937, 'keras': 2537, 'pandas': 27} layer_utils.py
{'numpy': 944, 'keras': 2537, 'pandas': 27} np_utils.py
{'numpy': 958, 'keras': 2537, 'pandas': 27} test_utils.py
{'numpy': 960, 'keras': 2540, 'pandas': 27} training_utils.py
{'numpy': 960, 'keras': 2540, 'pandas': 27} vis_utils.py
{'numpy': 960, 'keras': 2540, 'pandas': 27} __init__.py
{'numpy': 967, 'keras': 2540, 'pandas': 27} scikit_learn.py
{'numpy': 967, 'keras': 2540, 'pandas': 27} setup.py
{'numpy': 968, 'keras': 2550, 'pandas': 27} test_image_data_tasks.py
{'numpy': 986, 'keras': 2581, 'pandas': 27} test_temporal_data_tasks.py
{'numpy': 986, 'keras': 2607, 'pandas': 27} test_vector_data_tasks.py
{'numpy': 1018, 'keras': 2654, 'pandas': 27} activations_test.py
{'numpy': 1018, 'keras': 2732, 'pandas': 27} applications_test.py
{'numpy': 1026, 'keras

KeyboardInterrupt: 

In [44]:
a = 'from keras.layers.convolutional import (Convolution2D, MaxPooling2D'.split(' import ')[1]
a.replace('(','').replace(')','').replace(" ", "").split(',')

'from keras.models import Model'.split(' import ')[1].split()[-1]

'Model'

In [108]:
#repo_url = 'https://raw.githubusercontent.com/scikit-learn/scikit-learn/master/benchmarks/bench_20newsgroups.py'
repo_url = 'https://raw.githubusercontent.com/scikit-learn/scikit-learn/master/setup.py'
r = requests.get(repo_url,headers={"Accept":"application/vnd.github.mercy-preview+json"},
                 auth=HTTPBasicAuth(user, pw))

In [133]:
packages = ['numpy', 'distutils']

text = r.text
data = {}
for p in packages:
    if text.find(p) != -1:    # is package even in script at all?
        data[p] = 0
        
        # direct import, e.g. 'import numpy (as np)'
        string = 'import %s'%p
        loc = text.find(string)
        if loc != -1:       
            call_name = text[loc:].partition('\n')[0].split()[-1] # last word in line is call_name
            data[p] += text.count('%s.'%call_name)
        
        # indirect import, e.g. 'from numpy.X import vectorize (as vec)'
        string_from = 'from %s'%p
        loc_from = text.find(string_from)
        if loc_from != -1:  
            data[p] = 0
            while loc_from != -1: # can have multiple imports
                call_name = text[loc_from:].partition('\n')[0].split()[-1]
                data[p] += text.count('%s('%call_name)   # find function call
                loc_from = text.find(string_from, loc_from + 1) # get next instance, e.g. 'from numpy.X import b'
                
print(data)

numpy
distutils
{'numpy': 2, 'distutils': 1}


## Old/Extra Functions:
- scraping repos with a topic
- getting features for each repo

### First, scrape list of github repos with the following topics

In [113]:
topics = ['machine-learning']
max_requests = 4000

n_scraped = 0
for t in topics:
    link = 'https://api.github.com/search/repositories?q=topic:%s&per_page=100'%t
    f = open('repo_data/%s_repos.txt'%t, 'w')
    while link:
        r = requests.get(link, 
                     headers={"Accept":"application/vnd.github.mercy-preview+json"},
                     auth=HTTPBasicAuth(user, pw))
        if(r.ok):
            repos = json.loads(r.text or r.content)
            for item in repos['items']:
                f.write('%s\n'%item['url'])                
                n_scraped += 1 # number of repo urls scraped
        try:
            r_ = r.json()
            link = r.links['next']['url'] # get next page
            print(link)
        except:
            break

        if n_scraped >= max_requests:
            break
    
    f.close()
    print("Scraped %d repos for topic:'%s'"%(n_scraped, t))  # why is it only scraping 4x fewer than I want?

Scraped 86 repos for topic:'collision'


### Now, for each repo get the following Features:
- Topics
- Languages
- Popularity info - https://github.com/timqian/star-history - github star history  
- Lines of code - https://codetabs.com/count-loc/count-loc-online.html
- pep8 - https://github.com/vrana/pep8

In [115]:
topics = ['machine-learning']

for t in topics:
    repo_url_list = open('repo_data/%s_repos.txt'%t, 'r').read().splitlines() 
    data = {}
    for i, repo_url in enumerate(repo_url_list):
        try:
            r = requests.get(repo_url, 
                             headers={"Accept":"application/vnd.github.mercy-preview+json"},
                             auth=HTTPBasicAuth(user, pw))
            repoItem = json.loads(r.text or r.content)
            data[i] = {'topics': repoItem['topics'],
                       'language': repoItem['language'],
                       'stargazers': repoItem['stargazers_count'],
                       'watchers_count': repoItem['watchers_count'],
                       'forks_count': repoItem['forks_count'],
                       'url': repo_url}
        except:
            print('couldnt process github link %s'%repo_url)
    
    df = pd.DataFrame(data).T
    df.to_csv('repo_data/%s_data.csv'%t)

In [2]:
df = pd.read_csv('repo_data/machine-learning_data.csv') 

# need to make dtype of list be list
                 #dtype={'forks_count': np.int32, 'stargazers': np.int32, 'watchers_count': np.int32,
                 #      'language': 'str', 'url':'str', 'topics':'list'})
df.head()

,Unnamed: 0,forks_count,language,stargazers,topics,url,watchers_count
0,0,64002,C++,102092,"['deep-learning', 'deep-neural-networks', 'dis...",https://api.github.com/repos/tensorflow/tensor...,102092
1,1,11331,Python,30273,"['data-science', 'deep-learning', 'machine-lea...",https://api.github.com/repos/keras-team/keras,30273
2,2,14206,Python,28627,"['data-analysis', 'data-science', 'machine-lea...",https://api.github.com/repos/scikit-learn/scik...,28627
3,3,14940,C++,24409,"['deep-learning', 'machine-learning', 'vision']",https://api.github.com/repos/BVLC/caffe,24409
4,4,8338,Jupyter Notebook,22789,"['deep-learning', 'examples', 'machine-learnin...",https://api.github.com/repos/aymericdamien/Ten...,22789
